Использовать полученные знания в части обучения моделей для обучения и подбора параметров в задачах из предыдущих ДЗ на выбор (мфо, задача с тем возьмёт ли человек кредит или нет)
В частности использовать:

GaussianNB
LinearRegression + PolynomialFeatures.
При обучении использовать make_pipeline
GridSearchCV для поиска параметров


In [45]:
import numpy as np
import pandas as pd
import pandas_profiling # pip install pandas-profiling==3.6.5 ) последняя версия сломана
import matplotlib.pyplot as plt

from math import nan
from datetime import datetime 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score

MM  = MinMaxScaler(feature_range=(0,1))

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn import ReLU, Linear, BatchNorm1d, Dropout, Sequential, Module, BCEWithLogitsLoss, Sigmoid
import time
import gc

DATA_CSV_PATH = 'MFOcredit.csv'

### Загрузка, анализ и преобразование данных

Предварительная загрузка и отображение части данных

In [46]:
df_sample = pd.read_csv(DATA_CSV_PATH, encoding='cp1251', sep=';', nrows=3)

In [47]:
df_sample

,id,date_start,date_end,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,1,03-Jan-2013,12-Jan-2013,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет
1,2,03-Jan-2013,17-Jan-2013,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет
2,3,03-Jan-2013,17-Jan-2013,Мужской,25,Пропуск поля,Собственное,Холост,Да,Официальное,Кемеровская область север,Пропуск поля,Нет,Нет


Загрузка требуемых данных: для признаков явно указываем требуемые типы, признак id исключаем из набора

In [48]:
dtypes={
    'date_start': object,
    'date_end': object,
    'gender': object,
    'age': int,
    'auto': object,
    'housing': object,
    'marstatus': object,
    'regclient': object,
    'jobtype': object,
    'region': object,
    'credits': object,
    'children': object,
    'delinq60plus': object
    }

# Получить список признаков набора данных
col_list = df_sample.columns.tolist()

# Исключить признак 'id' из выборки
cols = col_list[1:]

df_raw = pd.read_csv(DATA_CSV_PATH, usecols = cols, dtype = dtypes, sep=';',encoding='cp1251')

# Заменить значения "пропуск поля" на NULL/NAN
#df_raw.replace('Пропуск поля', nan, inplace=True)

df_raw.head()

,date_start,date_end,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,03-Jan-2013,12-Jan-2013,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет
1,03-Jan-2013,17-Jan-2013,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет
2,03-Jan-2013,17-Jan-2013,Мужской,25,Пропуск поля,Собственное,Холост,Да,Официальное,Кемеровская область север,Пропуск поля,Нет,Нет
3,03-Jan-2013,17-Jan-2013,Женский,47,Пропуск поля,Собственное,Гражданский брак/женат/замужем,Да,Официальное,Кемеровская область север,Нет,Нет,Нет
4,03-Jan-2013,17-Jan-2013,Мужской,22,Нет,Арендуемое,Гражданский брак/женат/замужем,Нет,Официальное,Кемеровская область север,Да,Да,Нет


Заменить признаки дат начала (date_start) и окончания (date_end) периода займа на интегральный признак срока займа (loan_term)

In [49]:
date_start = pd.to_datetime(df_raw['date_start'])
date_end = pd.to_datetime(df_raw['date_end'])

df_raw.insert(0, 'start_year', date_start.dt.year) #.astype("category")) 
df_raw.insert(1, 'start_quart', date_start.dt.quarter) #.astype("category"))
df_raw.insert(2, 'start_month', date_start.dt.month) #.astype("category"))
df_raw.insert(3, 'start_day', date_start.dt.day) #.astype("category"))

df_raw.insert(4, 'end_year', date_end.dt.year) #.astype("category")) 
df_raw.insert(5, 'end_quart', date_end.dt.quarter) #.astype("category"))
df_raw.insert(6, 'end_month', date_end.dt.month) #.astype("category"))
df_raw.insert(7, 'end_day', date_end.dt.day) #.astype("category"))

df_raw.insert(8,'loan_term', (date_end - date_start).dt.days)

df_raw.drop(['date_start','date_end'], inplace=True, axis=1)
#df_raw.info()

Выделить и удалить строки-дубликаты

In [50]:
#df_duplicateRows = df_raw[df_raw.duplicated()]
#df_duplicateRows.shape

df_raw = df_raw.drop_duplicates(keep='first')
df_raw.shape

(35105, 20)

Выполнить анализ пропущенных значений

In [51]:
col_list = [i for i in df_raw.columns if df_raw[i].dtype.name=='object']
print('Процент значений "Пропуск поля" в предикторах:')
for col in col_list:
    if df_raw[col].str.contains(r"Пропуск поля").any():
        print(col)
        print(round((df_raw[col].value_counts()['Пропуск поля'])/
                    df_raw.shape[0]*100),'%')
    else:
        continue

Процент значений "Пропуск поля" в предикторах:
auto
40 %
housing
21 %
marstatus
21 %
jobtype
41 %
credits
3 %
children
2 %


Перевести кетегориальные признаки с бинарными значениями в числовую форму (Да=1 , Нет=0)

In [52]:
df_raw['gender'] = df_raw['gender'].apply(lambda x: 1 if x == 'Мужской' else 0 if x == 'Женский' else nan)
#df_raw['auto'] = df_raw['auto'].apply(lambda x: 1 if x == 'Да' else 0 if x == 'Нет' else nan)
df_raw['regclient'] = df_raw['regclient'].apply(lambda x: 1 if x == 'Да' else 0 if x == 'Нет' else nan)
#df_raw['jobtype'] = df_raw['jobtype'].apply(lambda x: 1 if x == 'Официальное' else 0 if x == 'Неофициальное' else nan)
df_raw['credits'] = df_raw['credits'].apply(lambda x: 1 if x == 'Да' else 0 if x == 'Нет' else nan)
df_raw['children'] = df_raw['children'].apply(lambda x: 1 if x == 'Да' else 0 if x == 'Нет' else nan)
df_raw['delinq60plus'] = df_raw['delinq60plus'].apply(lambda x: 1 if x == 'Да' else 0 if x == 'Нет' else nan)
df_raw['delinq60plus'].value_counts()

0    21614
1    13491
Name: delinq60plus, dtype: int64

### Замена отсутствующих значений категориальных признаков на наиболее часто встречающееся значение из общей группы

In [53]:
df_raw['children'].value_counts()

0.0    21069
1.0    13292
Name: children, dtype: int64

In [54]:
df_raw.groupby(['gender','age','region'])['children'].count().head(20)

gender  age  region                   
0       18   Новосибирская область          1
        19   Алтайский край                30
             Кемеровская область север     55
             Кемеровская область юг        62
             Красноярский край             11
             Новосибирская область         20
        20   Алтайский край                50
             Кемеровская область север     83
             Кемеровская область юг        74
             Красноярский край             12
             Новосибирская область         27
        21   Алтайский край                58
             Кемеровская область север    138
             Кемеровская область юг       108
             Красноярский край             26
             Новосибирская область         42
        22   Алтайский край                88
             Кемеровская область север    161
             Кемеровская область юг       158
             Красноярский край             36
Name: children, dtype: int64

In [55]:
df_raw.groupby(['gender','age','region'])['children'].count()

gender  age  region                   
0       18   Новосибирская область         1
        19   Алтайский край               30
             Кемеровская область север    55
             Кемеровская область юг       62
             Красноярский край            11
                                          ..
1       70   Красноярский край             1
        72   Кемеровская область юг        1
        73   Кемеровская область север     1
        74   Кемеровская область север     1
        75   Алтайский край                1
Name: children, Length: 530, dtype: int64

In [56]:
# Для признака "children" - значение из группы по признакам 'gender','age','region'
df_raw['children'] = df_raw.groupby(['gender','age','region'])['children'].transform(lambda x: x.fillna(x.mode()[0] if x.count()>0 else nan))
df_raw['children'].value_counts()

0.0    21670
1.0    13435
Name: children, dtype: int64

In [57]:
df_raw.groupby(['gender','age','region'])['children'].count()

gender  age  region                   
0       18   Новосибирская область         1
        19   Алтайский край               30
             Кемеровская область север    58
             Кемеровская область юг       62
             Красноярский край            11
                                          ..
1       70   Красноярский край             1
        72   Кемеровская область юг        1
        73   Кемеровская область север     1
        74   Кемеровская область север     1
        75   Алтайский край                1
Name: children, Length: 530, dtype: int64

In [58]:
df_raw['credits'] = df_raw.groupby(['gender','age','region'])['credits'].transform(lambda x: x.fillna(x.mode()[0] if x.count()>0 else nan))
df_raw['credits'].value_counts()

0.0    18659
1.0    16445
Name: credits, dtype: int64

In [59]:
df_raw['marstatus'] = df_raw.groupby(['gender','age','region','children'])['marstatus'].transform(lambda x: x.fillna(x.mode()[0] if x.count()>0 else nan))
df_raw['marstatus'].value_counts()

Гражданский брак/женат/замужем    14077
Холост                             8819
Пропуск поля                       7497
Разведен                           2870
Вдова/вдовец                       1842
Name: marstatus, dtype: int64

In [60]:
df_raw['housing'] = df_raw.groupby(['gender','age','region'])['housing'].transform(lambda x: x.fillna(x.mode()[0] if x.count()>0 else nan))
df_raw['housing'].value_counts()

Собственное              18067
Пропуск поля              7536
Живут с родителями        3172
Долевая собственность     2954
Арендуемое                2018
Муниципальное             1358
Name: housing, dtype: int64

Удалить оставшиеся "пропущенные" (NAN) значения

In [61]:
df_raw.dropna(inplace=True)
df_raw.shape

(35104, 20)

Разбиение признаков на группы числовых и категориальных признаков

In [62]:
num_columns = ['start_year', 'start_quart', 'start_month', 'start_day', 'end_year', 'end_quart', 'end_month', 'end_day', 'loan_term', 'age']
cat_columns = ['gender', 'housing', 'marstatus', 'regclient', 'region', 'credits', 'children', 'delinq60plus']
cat_unknowns = ['auto', 'jobtype']

In [63]:
df_num = df_raw[num_columns]
df_dummed = pd.get_dummies(df_raw[cat_columns])
df_unknowns = pd.get_dummies(df_raw[cat_unknowns])

data = pd.concat((df_num, df_dummed, df_unknowns), axis=1)
data.head()

,start_year,start_quart,start_month,start_day,end_year,end_quart,end_month,end_day,loan_term,age,...,region_Кемеровская область север,region_Кемеровская область юг,region_Красноярский край,region_Новосибирская область,auto_Да,auto_Нет,auto_Пропуск поля,jobtype_Неофициальное,jobtype_Официальное,jobtype_Пропуск поля
0,2013,1,1,3,2013,1,1,12,9,44,...,0,0,0,1,0,1,0,0,1,0
1,2013,1,1,3,2013,1,1,17,14,21,...,0,1,0,0,0,0,1,0,1,0
2,2013,1,1,3,2013,1,1,17,14,25,...,1,0,0,0,0,0,1,0,1,0
3,2013,1,1,3,2013,1,1,17,14,47,...,1,0,0,0,0,0,1,0,1,0
4,2013,1,1,3,2013,1,1,17,14,22,...,1,0,0,0,0,1,0,0,1,0


In [64]:
data.shape

(35104, 37)

---

Выполнить разбиение исходной выборки данных на 2 подмножества (обучение и тестирование)

In [65]:
y = data['delinq60plus']
X = data.drop(['delinq60plus'], axis=1)

In [66]:
y.value_counts()

0    21613
1    13491
Name: delinq60plus, dtype: int64

In [67]:
X=X.to_numpy()
y=y.to_numpy()

In [68]:
X

array([[2.013e+03, 1.000e+00, 1.000e+00, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.013e+03, 1.000e+00, 1.000e+00, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.013e+03, 1.000e+00, 1.000e+00, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [2.014e+03, 1.000e+00, 1.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [2.014e+03, 1.000e+00, 1.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [2.014e+03, 1.000e+00, 1.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00]])

In [69]:
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [70]:
from sklearn.model_selection import train_test_split
RANDOM_STATE = 17
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size= 0.2, random_state=RANDOM_STATE, stratify=y_train)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(21062, 36)
(21062,)
(8776, 36)
(8776,)
(5266, 36)
(5266,)


In [71]:
X_train = MM.fit_transform(X_train)
X_val = MM.fit_transform(X_val)
X_test = MM.fit_transform(X_test)

Создаем загрузчик

In [72]:
class Data(Dataset):

    def __init__(self, X, y):

        self.X = torch.tensor(X, dtype = torch.float32)
        self.y = torch.tensor(y, dtype = torch.float32)
          
    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.X)

In [73]:
train_data = Data(X_train, y_train)
test_data = Data(X_test, y_test)
val_data = Data(X_val, y_val)

Формируем нейронную сеть

In [74]:
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(36,100)
        self.fc2 = nn.Linear(100,50)
        self.fc3 = nn.Linear(50,1)
        
        self.relu = ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [75]:
def binary_acc(y_pred, y_true):
    y_pred_val = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = torch.eq(y_pred_val, y_true).sum().float()
    acc = correct_results_sum/y_true.shape[0]

    return acc

In [76]:
simplenet = SimpleNet()

In [77]:
simplenet

SimpleNet(
  (fc1): Linear(in_features=36, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
)

In [78]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.0001)

Скопируйте модель на графический процессор, если таковой имеется.

In [79]:
if torch.cuda.is_available():
    device = torch.device("cuda:0") 
else:
    device = torch.device("cpu")

simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=36, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
)

In [80]:
torch.cuda.is_available()

True

Пишем функцию обучения

Тренировка 

Обучает модель, при необходимости копируя пакеты на графический процессор, вычисляя потери, оптимизируя сеть и выполняя проверку для каждой эпохи.

In [81]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20,device="cuda:0"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        train_acc = 0.0
        val_loss = 0.0
        val_acc = 0.0
        
        model.train()
        
        for batch in train_loader:
            optimizer.zero_grad()      
            X, y = batch
            X = X.to(device)
            y = y.to(device)
           
            y_pred = model(X)
            loss = loss_fn(y_pred, y.unsqueeze(1))
            acc = binary_acc(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
            train_acc += acc.item()
            
        training_loss /= len(train_loader)
        train_acc /= len(train_loader)
        
        model.eval()
                
        for batch in val_loader:
          
            X, y = batch
            X = X.to(device)
            y = y.to(device)
           
            y_pred = model(X)
            loss = loss_fn(y_pred, y.unsqueeze(1))
            acc = binary_acc(y_pred, y.unsqueeze(1))
            val_loss += loss.item()
            val_acc += acc.item()
            
        val_loss /= len(val_loader)
        val_acc /= len(val_loader)
        
        print('Epoch: {}, Training Loss: {:.2f}, accuracy = {:.5f}, Validation Loss: {:.2f}, accuracy = {:.5f}'.format(epoch, training_loss, train_acc, val_loss, val_acc))


In [82]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64)

In [83]:
loss_fn = torch.nn.BCEWithLogitsLoss()

In [84]:
train(simplenet,optimizer,loss_fn,train_loader,val_loader, epochs=10)

Epoch: 1, Training Loss: 0.65, accuracy = 0.60347, Validation Loss: 0.61, accuracy = 0.62642
Epoch: 2, Training Loss: 0.56, accuracy = 0.69818, Validation Loss: 0.53, accuracy = 0.72360
Epoch: 3, Training Loss: 0.51, accuracy = 0.74265, Validation Loss: 0.51, accuracy = 0.73546
Epoch: 4, Training Loss: 0.49, accuracy = 0.74890, Validation Loss: 0.50, accuracy = 0.74092
Epoch: 5, Training Loss: 0.49, accuracy = 0.75268, Validation Loss: 0.50, accuracy = 0.74224
Epoch: 6, Training Loss: 0.48, accuracy = 0.75391, Validation Loss: 0.50, accuracy = 0.74243
Epoch: 7, Training Loss: 0.48, accuracy = 0.75496, Validation Loss: 0.49, accuracy = 0.74431
Epoch: 8, Training Loss: 0.48, accuracy = 0.75571, Validation Loss: 0.49, accuracy = 0.74412
Epoch: 9, Training Loss: 0.48, accuracy = 0.75604, Validation Loss: 0.49, accuracy = 0.74450
Epoch: 10, Training Loss: 0.48, accuracy = 0.75666, Validation Loss: 0.49, accuracy = 0.74628


In [85]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

Проверяем на тестовом наборе

In [86]:
test_loss = 0.0
test_acc = 0.0
model_test = simplenet.eval()
loss_fn = torch.nn.BCEWithLogitsLoss()

In [87]:
for batch in test_loader:
    optimizer.zero_grad()      
    X, y = batch
    X = X.to(device)
    y = y.to(device)

    y_pred = model_test(X)
        
    loss = loss_fn(y_pred, y.unsqueeze(1))
    acc = binary_acc(y_pred, y.unsqueeze(1))
            
    loss.backward()
    optimizer.step()
            
    test_loss += loss.item()
    test_acc += acc.item()
            
test_loss /= len(test_loader)
test_acc /= len(test_loader)
print('Test Loss: {:.2f}, Test accuracy = {:.5f}'.format(test_loss, test_acc))

Test Loss: 0.47, Test accuracy = 0.75600
